# s̅ derivation
 s̅<sub>i,j</sub> is the value needed for calculating sensitivities
 The formula for it is the following **(from equation 13 in the Yue et. al. paper)**
 
 ```
  s̅_i,j = (dx_i)/(dtheta_j) * theta_j/x_i 
 ```
 
 where `i` is the i<sup>th</sup> state variable and `j` is j<sup>th</sup> parameter value
 
 
 x<sub>i</sub> is the i<sup>th</sup> entry in the state matrix X which is defined as
 ```
 X = [x1, x2, ...]
 ```
 
 theta<sub>j</sub> is the j<sup>th</sup> entry in the theta matrix which is defined as 
 ```
 theta = [k1, k2, ...]
 ```
 
 ### Derivation
 The symbol library `sympy` is good for derivatives. Heres a link to some examples:
 
 https://towardsdatascience.com/taking-derivatives-in-python-d6229ba72c64
 
 #### Example

In [2]:
import sympy as sp
x = sp.Symbol('x')
y = sp.diff('x**2', x)
print(y)

2*x


#### Example algebra

In [3]:
eq = 'x+y'
neweq = sp.Mul(sp.sympify(eq), sp.sympify('v'))
print(neweq)
neweq2 = sp.Mul(neweq, sp.sympify('c**-1'))
print(neweq2)

v*(x + y)
v*(x + y)/c


## Perform the derivation
### Load state space model

In [12]:
import sympy as sp

ssm_file = '/Users/zacharymcgrath/Documents/School/Spring 2020/Dynamic Systems in Biology/dynamic_models_in_bio_final_project/state_space_derivation.txt'
eqts = []
xs = []
thetas = []

theta_char = 'k'
extras = ['(', ')', '[', ']']

def get_symbols(string):
    symbols = []
    nopows = string.split('**')
    for nopow in nopows:
        nodivs = nopow.split('/')
        for nodiv in nodivs:
            nomults = nodiv.split('*')
            for nomult in nomults:
                noadds = nomult.split('+')
                for noadd in noadds:
                    nosub = noadd.split('-')
                    for s in nosub:
                        for ext in extras:
                            s = s.replace(ext, '')
                        symbols.append(s)
    return symbols

with open(ssm_file, 'r') as o:
    for line in o:
        x = line.split('=')[0].split('(')[0]
        if x not in xs:
            xs.append(x)
        eqt = line.split('=')[1].strip()
        symbols = get_symbols(eqt)
        for s in symbols:
            if theta_char in s and s not in thetas:
                thetas.append(s)
                
        eqts.append((x, eqt))
    
thetas.sort(key=lambda x: int(x[1:]))   
print(xs)
print(thetas)

['x1', 'x2', 'x3', 'x4', 'x5', 'x6', 'x7', 'x8', 'x9', 'x10', 'x11', 'x12', 'x13', 'x14', 'x15', 'x16', 'x17', 'x18', 'x19', 'x20', 'x21', 'x22', 'x23', 'x24']
['k1', 'k2', 'k3', 'k4', 'k5', 'k6', 'k7', 'k8', 'k9', 'k10', 'k11', 'k12', 'k13', 'k14', 'k15', 'k16', 'k17', 'k18', 'k19', 'k20', 'k21', 'k22', 'k23', 'k24', 'k25', 'k26', 'k27', 'k28', 'k29', 'k30', 'k31', 'k32', 'k33', 'k34', 'k35', 'k36', 'k37', 'k38', 'k39', 'k40', 'k41', 'k42', 'k43', 'k44', 'k45', 'k46', 'k47', 'k48', 'k49', 'k50', 'k51', 'k52', 'k53', 'k54', 'k55', 'k56', 'k57', 'k58', 'k59', 'k60', 'k61', 'k62', 'k63']


### Find the derivatives
The derivative list will by `i` by `j` in size, with `i` being the number of `x` variables and `j` being the number of `theta` variables

In [13]:
derivatives = [['' for j in range(len(thetas))] for i in range(len(xs))]
for i in range(len(xs)):
    for j in range(len(thetas)):
        derivatives[i][j] = sp.diff(eqts[i][1], thetas[j])
print(eqts[0][1])
print(thetas[0])
print(derivatives[0][0])

-k1*x1*x2+k2*x3-k34*x10*x1+k35*x8+k36*x22-k37*x1-k38*x1+k39*x16
k1
-x1*x2


### Calculate s̅

In [14]:
s_bar = [['' for j in range(len(thetas))] for i in range(len(xs))]
for i in range(len(xs)):
    for j in range(len(thetas)):
        s_bar[i][j] = sp.Mul((sp.Mul(sp.sympify(derivatives[i][j]) * sp.sympify(thetas[j]))), sp.sympify('{}**-1'.format(xs[i])))
        
print(derivatives[0][0])
print(s_bar[0][0])

-x1*x2
-k1*x2


In [15]:
print(eqts[6][0] + ' = ' + eqts[6][1])
print(derivatives[6][5])
print(s_bar[6][5])

x7 = +k5*x6*x2-k6*x7-k18*x7-k58*x10*x7+k59*x14+k60*x21
-x7
-k6


## Dump to a file

In [17]:
import pickle

pickle_file = '/Users/zacharymcgrath/Documents/School/Spring 2020/Dynamic Systems in Biology/dynamic_models_in_bio_final_project/sensitivity_matrix'
pickle.dump(s_bar, open(pickle_file, 'wb'))